In [3]:
import glob
import os
import json
from PIL import Image, ImageDraw
from shapely.geometry import Polygon
import numpy as np
import pickle

# Convert Labelme JSON files to one coco json file

In [4]:
class labelme2coco(object):
    def __init__(self,labelme_json=[],save_json_path='./new.json',classes={}):
        '''
        :param labelme_json: all the labelme json files
        :param save_json_path: the dir of saved json files
        '''
        self.labelme_json=labelme_json
        self.save_json_path=save_json_path
        self.images=[]
        self.categories=[]
        self.annotations=[]
        # self.data_coco = {}
        self.__classes__ = classes
        self.label=['background']
        self.annID=1
        #self.height=4000
        #self.width=6000

        self.save_json()

    def data_transfer(self):
        null_num = 0
        for num,json_file in enumerate(self.labelme_json):
            #if num % 10 != 0:
            #    continue
            with open(json_file,'r') as fp:

                data = json.load(fp)  # load the json files
                if len(data['shapes']) == 0:
                    print(json_file)
                    null_num += 1
                    continue
                
                self.images.append(self.image(data,num))
                for shapes in data['shapes']:
                    #full_label=shapes['label']
                    label=shapes['label'].split('_')[0]
                    #需要根据 label 的具体格式，调整这几行代码
                    #if full_label.find("spike") != -1:
                        #label = "spike"
                    #if full_label.find("barley") != -1:
                        #label = "barley"
                    if label not in self.label:
                        self.categories.append(self.create_category(label))
                        self.label.append(label)
                    
                    points=shapes['points']
                    if len(points) < 3:
                        continue
                    self.annotations.append(self.annotation(points,label,num))
                    self.annID+=1
        
        print("Total convert {} json files".format(num + 1- null_num))

    def image(self,data,num):
        image={}
        #image size is 4000 * 6000
        #img = utils.img_b64_to_arr(data['imageData']) 
        #height, width = img.shape[:2]
        #img = None
        
        image['height']= data['imageHeight']
        image['width'] = data['imageWidth']
        image['id']=num+1
        image['file_name'] = data['imagePath'].split('/')[-1]

        self.height=data['imageHeight']
        self.width=data['imageWidth']

        return image

    #根据 new label 生成一个新的 Category
    def create_category(self,label):
        category={}
        category['supercategory'] = "wheat" #这个值无实际用处

        category['id'] = self.__classes__[label] #len(self.label) #self.label 中，初始有个background，所以直接取list的 length
        category['name'] = label
        return category

    def annotation(self,points,label,num):
        annotation={}
        annotation['segmentation']=[np.asarray(points).flatten().tolist()]
        annotation['iscrowd'] = 0
        annotation['image_id'] = num+1

        annotation['bbox'] = list(map(float,self.getbbox(points)))

        annotation['category_id'] = self.getcatid(label)
        annotation['id'] = self.annID

        # Get the area value
        poly = Polygon(points)
        annotation['area']=round(poly.area, 6)
        return annotation

    def getcatid(self,label):
        for category in self.categories:
            if label == category['name']:
                return category['id']
        return -1

    '''
    def getbbox(self,points):

        polygons = points
        mask = self.polygons_to_mask([self.height,self.width], polygons)
        return self.mask2box(mask)
    '''
    def getbbox(self, points):
        polygons =  points
        #px = [x for x, y in polygons]
        #py = [y for x, y in polygons]
        px = []
        py = []
        im_height = self.height
        im_width = self.width

        for x,y in polygons:
            if x < 0:
                x = 0
            if y < 0 :
                y = 0
            if x >= im_width:
                x = im_width -1
            if y >= im_height:
                y = im_height -1
            px.append(int(x))
            py.append(int(y))

        x1 = np.min(px)
        y1 = np.min(py)
        x2 = np.max(px)
        y2 = np.max(py)
        return [x1, y1, x2-x1, y2-y1] # [x1,y1,w,h] 对应COCO的bbox格式

    def mask2box(self, mask):
        '''从mask反算出其边框
        mask：[h,w]  0、1组成的图片
        1对应对象，只需计算1对应的行列号（左上角行列号，右下角行列号，就可以算出其边框）
        '''
        # np.where(mask==1)
        index = np.argwhere(mask == 1)
        rows = index[:, 0]
        clos = index[:, 1]
        # 解析左上角行列号
        left_top_r = np.min(rows)  # y
        left_top_c = np.min(clos)  # x

        # 解析右下角行列号
        right_bottom_r = np.max(rows)
        right_bottom_c = np.max(clos)

        # return [(left_top_r,left_top_c),(right_bottom_r,right_bottom_c)]
        # return [(left_top_c, left_top_r), (right_bottom_c, right_bottom_r)]
        # return [left_top_c, left_top_r, right_bottom_c, right_bottom_r]  # [x1,y1,x2,y2]
        return [left_top_c, left_top_r, right_bottom_c-left_top_c, right_bottom_r-left_top_r]  # [x1,y1,w,h] 对应COCO的bbox格式

    def polygons_to_mask(self,img_shape, polygons):
        mask = np.zeros(img_shape, dtype=np.uint8)
        mask = Image.fromarray(mask)
        xy = list(map(tuple, polygons))
        ImageDraw.Draw(mask).polygon(xy=xy, outline=1, fill=1)
        mask = np.array(mask, dtype=bool)
        return mask

    def data2coco(self):
        data_coco={}
        data_coco['images']=self.images
        data_coco['categories']=self.categories
        data_coco['annotations']=self.annotations
        return data_coco

    def save_json(self):
        self.data_transfer()
        self.data_coco = self.data2coco()

        json.dump(self.data_coco, open(self.save_json_path, 'w'), indent=4)  # indent=4

In [5]:
# Barley seeds malt classes:
malt_classes = {"up":1,"down":2}

In [6]:
# change 1，2 ，3，4，acrospire to 1
seeds_malt_classes = {"0":1,"1":1, "2":1, "3":1, "4":1, "acrospire":1}

In [7]:
train_1_jsons = glob.glob(r'E:\Temp\barley_malt\single_seeds\mini_train_set_single1\*.json')
train_2_jsons = glob.glob(r'E:\Temp\barley_malt\single_seeds\mini_train_set_single2\*.json')
train_3_jsons = glob.glob(r'E:\Temp\barley_malt\single_seeds\mini_train_set_single3\*.json')
train_jsons = train_1_jsons + train_2_jsons + train_3_jsons
len(train_jsons)

1276

In [8]:
# train json
labelme2coco(train_jsons,'./barley_malt/single_seeds/seeds_malt_train.json', seeds_malt_classes)

Total convert 1276 json files


In [12]:
# test
test_jsons = glob.glob(r'E:\Temp\barley_malt\single_seeds\test_set_instSegm_oneclass\*.json')
len(test_jsons)

312

In [14]:
test_jsons

['E:\\Temp\\barley_malt\\single_seeds\\test_set_instSegm_oneclass\\1102_10_001.json',
 'E:\\Temp\\barley_malt\\single_seeds\\test_set_instSegm_oneclass\\1102_10_003.json',
 'E:\\Temp\\barley_malt\\single_seeds\\test_set_instSegm_oneclass\\1102_10_004.json',
 'E:\\Temp\\barley_malt\\single_seeds\\test_set_instSegm_oneclass\\1102_10_009.json',
 'E:\\Temp\\barley_malt\\single_seeds\\test_set_instSegm_oneclass\\1102_10_019.json',
 'E:\\Temp\\barley_malt\\single_seeds\\test_set_instSegm_oneclass\\1102_10_021.json',
 'E:\\Temp\\barley_malt\\single_seeds\\test_set_instSegm_oneclass\\1102_10_022.json',
 'E:\\Temp\\barley_malt\\single_seeds\\test_set_instSegm_oneclass\\1102_10_024.json',
 'E:\\Temp\\barley_malt\\single_seeds\\test_set_instSegm_oneclass\\1102_10_025.json',
 'E:\\Temp\\barley_malt\\single_seeds\\test_set_instSegm_oneclass\\1102_10_027.json',
 'E:\\Temp\\barley_malt\\single_seeds\\test_set_instSegm_oneclass\\1102_10_032.json',
 'E:\\Temp\\barley_malt\\single_seeds\\test_set_instSe

In [13]:
labelme2coco(test_jsons,'./barley_malt/single_seeds/seeds_malt_test.json', seeds_malt_classes)

KeyError: 'shapes'

In [4]:
# Convert Barley Malt train json files

labelme_json=glob.glob(r'e:\temp\barley_malt\train\*.json')
labelme2coco(labelme_json,'./barley_malt/barley_malt_train.json', malt_classes)

Total convert 55 json files


In [21]:
# Convert Barley Malt test json files

labelme_json=glob.glob(r'e:\temp\barley_malt\test\*.json')
labelme2coco(labelme_json,'./barley_malt/barley_malt_test.json', malt_classes)

Total convert 6 json files


In [ ]:
# Convert train data set json files
labelme_json=glob.glob(r'e:\temp\train\*.json')
labelme2coco(labelme_json,'./wheat2019_train.json')

In [ ]:
# Convert test data set json files
labelme_json=glob.glob(r'E:\Temp\val\good_imgs\*.json')
labelme2coco(labelme_json,'./wheat2019_val.json')

In [ ]:
labelme_json=glob.glob(r'e:\temp\val\*.json')
labelme2coco(labelme_json,'./wheat2019_val_mini.json')

In [ ]:
# Convert test data set json files
labelme_json=glob.glob(r'e:\temp\val\*.json')
labelme2coco(labelme_json,'./wheat2019_val.json')

In [ ]:
labelme_json = glob.glob(r'E:\mytmp\g_drv\val_json\*.json')
labelme2coco(labelme_json,'./wheat2019_spikes_val.json')

In [ ]:
labelme_json = glob.glob(r'E:\mytmp\g_drv\train_json\*.json')
labelme2coco(labelme_json,'./wheat2019_spikes_train.json')

In [ ]:
# Convert train data set json files
#labelme_json=glob.glob(r'e:\temp\train\*.json')
labelme_json=glob.glob(r'E:\Temp\train_class\wheat good\*.json')
labelme2coco(labelme_json,'./wheat2019_disease_train.json')

In [ ]:
# Convert test data set json files
#labelme_json=glob.glob(r'e:\temp\val\*.json')
labelme_json=glob.glob(r'E:\Temp\val_class\wheat good\*.json')
labelme2coco(labelme_json,'./wheat2019_disease_val.json')

In [ ]:
labelme_json=glob.glob(r'E:\Temp\val\no_beard\*.json')
labelme2coco(labelme_json,'./wheat2019_no_awn_val.json')

In [ ]:
# convert train data set json files
labelme_json=glob.glob(r'e:\temp\anchor_test\*.json')
labelme2coco(labelme_json,'./wheat2019_anchor_test.json')

In [ ]:
#convert barley images
labelme_json=glob.glob(r'E:\Temp\train\barley\good\*.json')
labelme2coco(labelme_json,'./2019_barley_train.json')

In [ ]:
# 2020 Barley iter 1 images
json_file1 = glob.glob(r'E:\2020_barley\barley_less\iter1\*.json')
json_file2 = glob.glob(r'E:\2020_barley\barley_more\iter1\*.json')
labelme_json = json_file1 + json_file2
labelme2coco(labelme_json,'./2020_barley_iter1_train.json')

In [3]:
# 2020 Barley iter 1 images
json_file = glob.glob(r'E:\2020_barley\barley_more\iter1\*.json')
labelme_json = json_file
labelme2coco(labelme_json,'./2020_barley_iter1_more_infer.json')

Total convert 53 json files


In [3]:
#2020 barley iter1  less part train images
json_file1 = glob.glob(r'E:\2020_barley\barley_less\iter1\train\*.json')
labelme_json = json_file1
labelme2coco(labelme_json,'./2020_barley_iter1_less_train.json')

Total convert 212 json files


In [4]:
#2020 barley iter1  less part val images
json_file1 = glob.glob(r'E:\2020_barley\barley_less\iter1\val\*.json')
labelme_json = json_file1
labelme2coco(labelme_json,'./2020_barley_iter1_less_val.json')

Total convert 92 json files


In [3]:
#2020 barley iter1  less part train + val images together for training
json_file1 = glob.glob(r'E:\2020_barley\barley_less\iter1\train\*.json')
json_file2 = glob.glob(r'E:\2020_barley\barley_less\iter1\val\*.json')
labelme_json = json_file1 + json_file2
labelme2coco(labelme_json,'./2020_barley_iter1_less_train_val.json')

Total convert 304 json files


In [6]:
## 2020 barley iter2 infer jsons
json_file = pickle.load(open("E:/2020_barley/iter2_infer_jsons.pkl", "rb"))
labelme_json = json_file
labelme2coco(labelme_json,'./2020_barley_iter2_infer.json')

Total convert 1454 json files


In [5]:
len(json_file)

1454

In [3]:
# create fake json for wheat 2020 infer
labelme_json=glob.glob("E:/wheat2020/wheat_infer/*.json")
labelme2coco(labelme_json,'./2020_wheat_fake_infer.json')

Total convert 261 json files


In [ ]:
# Using 2019 barley images val part for test
labelme_json=glob.glob(r"E:\Temp\val\barley\*.json")
labelme2coco(labelme_json,'./2019_barley_val.json')

In [ ]:
# Using wheat 2019 wheat and 2020 barley iter1 less part for training
json_file1 = glob.glob(r'E:\2020_barley\barley_less\iter1\*.json')
json_file2 = glob.glob(r'e:\temp\train\*.json')
labelme_json = json_file1 + json_file2
labelme2coco(labelme_json,'./2020_wheat_barley_iter1_less_train.json')

In [ ]:
#convert no_beard images
labelme_json=glob.glob(r'E:\mytmp\no_bear_spikes\*.json')
labelme2coco(labelme_json,'./2019_no_beard_test.json')

In [ ]:
name = "dise_spike_1"

if name.find("spike") != -1:
    print ("spike")

In [ ]:
import json
import os

#dataset_dicts = []
with open("wheat2019_val_dt1.json") as f_dt1:
    dt1 = json.load(f_dt1)
    

with open("wheat2019_val_dt2.json") as f_dt2:
    dt2 = json.load(f_dt2)
    
for image_dt1 in dt1["images"]:
    image_dt1_file_name = "/home/ceyang/coco/wheat2019_val/" + image["file_name"]
    image_dt1_id = image["id"]

### Split spike images ###

In [ ]:
labelme_json=glob.glob(r'E:\img_split\train_imgs\*.json')
labelme2coco(labelme_json,'./wheat2019_split_train.json')

In [ ]:
labelme_json=glob.glob(r'E:\img_split\val_jsons\*.json')
labelme2coco(labelme_json,'./wheat2019_split_val.json')

### HSV spike img jsons

In [ ]:
labelme_json=glob.glob(r'E:\disease_segm\train_hsv_json\*.json')
labelme2coco(labelme_json,'./wheat2019_hsv_spike_train.json')

In [ ]:
labelme_json=glob.glob(r'E:\disease_segm\val_hsv_json\*.json')
labelme2coco(labelme_json,'./wheat2019_hsv_spike_val.json')

In [ ]:
labelme_json=glob.glob(r'E:\2020_barley\barley_imgs\*.json')
labelme2coco(labelme_json,'./barley2020_fake_val.json')